# Explicación de la notebook 7

En esta notebook se utiliza el modelo de LIGHT GBM, un modelo basado en arboles de decisión. Primero se hizo una corrida con los parametros default del regresor excepto por algunos ingresados manualmente, que son n_estimators = 4000 learning_rate=0.05 y min_child_samples=5. Con estos parametros, se consiguio un score de 0.11412. Luego, se hizo un tunning de distintos hyperparametros, hasta llegar al mejor score, de 0.10715, que resulto también el mejor score de clasificación de todo el trabajo práctico. 

Para conseguir este score, se utilizaron los siguientes parametros:

params = {"objective" : "rmse",
          "boosting" : "gbdt", 
          "metric" : "rmse",
          "num_iterations" : 15148,
          "top_k" : 30, 
          "max_depth" : max_depth, 
          "num_leaves" : num_leaves, 
          "min_data_in_leaf" : min_data_in_leaf, 
          "learning_rate" : 0.02,
          "bagging_fraction" : 0.7, 
          "bagging_seed" : 3,
          "bagging_freq" : 5, 
          "feature_fraction" : 0.5, 
          "num_threads" : 4
         }
         
dataset_params = {"max_bin" : 200, 
                  "min_data_in_bin" : 3} 

Cabe destacar que se probó utilizar , con los parametros defalult (n_estimators = 4000 learning_rate=0.05 y min_child_samples=5), y se consiguio un resultado casi igual,al de sin embedding y parametros default, con score de 0.11555.

### SUBMISSIONS CON ESTOS SCORES: 

- Parametros default sin embedding: submisionn_True-5-4000-0.05.csv (score = 0.11412)

- Parametros default con embedding: submision_LGBM_embedd_True-5-4000-0.05-8-800-csv (Score = 0.11555)

- Parametros cambiados sin embedding: submision_LGBM_muchos_param_True-5-4000-0.02-8-800-.csv (score = 0.10715) EL MEJOR






# Código 

In [3]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

https://www.youtube.com/watch?v=1-NYPQw5THU&feature=youtu.be

In [4]:
import pandas as pd
import numpy as np
import datetime
from pandas_summary import DataFrameSummary

In [5]:
df = pd.read_feather('train_normalized_data.fth')
df_test = pd.read_feather('test_normalized_data.fth')

In [6]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw']

# cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'State'] 'StateHoliday_bool_fw', 'StateHoliday_bool_bw'


In [7]:
#cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
#            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
#            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw', 'Promo', 'SchoolHoliday',
#            'StateHoliday_bool_fw', 'StateHoliday_bool_bw']

In [8]:
#contin_vars = ['CompetitionDistance', 
#              'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 
#               'Precipitationmm', 'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 
#               'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 
#               'trend_DE', 'AfterStateHoliday_bool', 'BeforeStateHoliday_bool']


In [9]:
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE', 'Promo', 'SchoolHoliday']

In [10]:
#contin_vars = ['CompetitionDistance', 
#   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
#   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
#   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
#    'Promo', 'SchoolHoliday']
# contin_vars = [] 'AfterStateHoliday_bool', 'BeforeStateHoliday_bool'

In [11]:
#pip install lightgbm

In [13]:
from lightgbm import LGBMRegressor


In [14]:
y_out_columns = ['Sales']

In [15]:
df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')

Cantidad en val: 30188, porcentaje: 0.9642465458145908


In [16]:
X_train = df_train[cat_vars + contin_vars]
X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

In [17]:
X_train.shape, X_val.shape

((814150, 36), (30188, 36))

In [18]:
log_output = True
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    y_mean = df_train[y_out_columns].mean().values
    y_std = df_train[y_out_columns].std().values
    y_train = (df_train[y_out_columns].values - y_mean)/y_std
    y_val = (df_val[y_out_columns].values - y_mean)/y_std

C:\Users\pedro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


In [19]:
frames = [X_train, X_val]
X = pd.concat(frames)

y = np.concatenate((y_train, y_val), axis=0)


In [20]:
"""
study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, X, y)
study.optimize(func, n_trials=20)
"""

'\nstudy = optuna.create_study(direction="minimize", study_name="LGBM Classifier")\nfunc = lambda trial: objective(trial, X, y)\nstudy.optimize(func, n_trials=20)\n'

## EMBEDDINGS

In [21]:
"""
cat_var_dict = {'Store': 50, 'DayOfWeek': 2, 'Year': 2, 'Month': 2,
'Day': 10, 'StateHoliday': 2, 'CompetitionMonthsOpen': 2,
'Promo2Weeks': 1, 'StoreType': 2, 'Assortment': 3, 'PromoInterval': 3,
'CompetitionOpenSinceYear': 4, 'Promo2SinceYear': 4, 'State': 6,
'Week': 25, 'Events': 4, 'Promo_fw': 1,
'Promo_bw': 1, 'StateHoliday_bool_fw': 1,
'StateHoliday_bool_bw': 1, 'SchoolHoliday_fw': 1,
'SchoolHoliday_bw': 1,
'Promo': 2, 'SchoolHoliday': 2,
}
"""

"\ncat_var_dict = {'Store': 50, 'DayOfWeek': 2, 'Year': 2, 'Month': 2,\n'Day': 10, 'StateHoliday': 2, 'CompetitionMonthsOpen': 2,\n'Promo2Weeks': 1, 'StoreType': 2, 'Assortment': 3, 'PromoInterval': 3,\n'CompetitionOpenSinceYear': 4, 'Promo2SinceYear': 4, 'State': 6,\n'Week': 25, 'Events': 4, 'Promo_fw': 1,\n'Promo_bw': 1, 'StateHoliday_bool_fw': 1,\n'StateHoliday_bool_bw': 1, 'SchoolHoliday_fw': 1,\n'SchoolHoliday_bw': 1,\n'Promo': 2, 'SchoolHoliday': 2,\n}\n"

In [22]:
#from tensorflow.keras.models import Model
#from tensorflow.keras.callbacks import ModelCheckpoint
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.layers import Embedding, Input, Flatten, Concatenate, Dense, BatchNormalization, Activation, LeakyReLU, Dropout
#from tensorflow.keras.regularizers import l2

In [23]:
"""def get_cat_vars_model(cat_vars, uniques, cat_var_dict):
    cat_vars_embed_outs = []
    cat_var_inputs = []
    for cat_var in cat_vars:
        cat_var_in = Input(shape=(1,), name=f"{cat_var}_input")
        cat_var_inputs.append(cat_var_in)
        embed_out = Embedding(uniques[cat_var][0], cat_var_dict[cat_var], name=f'{cat_var}_Embed')(cat_var_in)
        flatten_out = Flatten(name=f"{cat_var}_flat")(embed_out)
        cat_vars_embed_outs.append(flatten_out)
    return cat_var_inputs, cat_vars_embed_outs

def get_cont_vars_input(contin_vars, dense_layer=False):
    cont_vars_inputs = []
    cont_vars_outputs = []
    for cont_var in contin_vars:
        cont_var_in = Input(shape=(1,), name=f"{cont_var}_input")
        cont_vars_inputs.append(cont_var_in)
        if dense_layer:
            cont_var_out = Dense(1, name=f"{cont_var}_input", activation = 'linear')(cont_var_in)
            cont_vars_outputs.append(cont_var_out)
        else:
            cont_vars_outputs.append(cont_var_in)
    return cont_vars_inputs, cont_vars_outputs
    """

'def get_cat_vars_model(cat_vars, uniques, cat_var_dict):\n    cat_vars_embed_outs = []\n    cat_var_inputs = []\n    for cat_var in cat_vars:\n        cat_var_in = Input(shape=(1,), name=f"{cat_var}_input")\n        cat_var_inputs.append(cat_var_in)\n        embed_out = Embedding(uniques[cat_var][0], cat_var_dict[cat_var], name=f\'{cat_var}_Embed\')(cat_var_in)\n        flatten_out = Flatten(name=f"{cat_var}_flat")(embed_out)\n        cat_vars_embed_outs.append(flatten_out)\n    return cat_var_inputs, cat_vars_embed_outs\n\ndef get_cont_vars_input(contin_vars, dense_layer=False):\n    cont_vars_inputs = []\n    cont_vars_outputs = []\n    for cont_var in contin_vars:\n        cont_var_in = Input(shape=(1,), name=f"{cont_var}_input")\n        cont_vars_inputs.append(cont_var_in)\n        if dense_layer:\n            cont_var_out = Dense(1, name=f"{cont_var}_input", activation = \'linear\')(cont_var_in)\n            cont_vars_outputs.append(cont_var_out)\n        else:\n            cont

In [24]:
#uniques = DataFrameSummary(df[cat_vars]).summary().loc[['uniques']]

In [25]:
#cat_var_inputs, cat_vars_embed_outs = get_cat_vars_model(cat_vars, uniques, cat_var_dict)
#cont_vars_inputs,  cont_vars_outs= get_cont_vars_input(contin_vars)

## CLASIFICACION

In [29]:
min_child_samples=5
min_data_in_leaf = 20
n_estimators = 4000
learning_rate=0.02
max_depth = 8
num_leaves = 800

params = {"objective" : "rmse",
          "boosting" : "gbdt", 
          "metric" : "rmse",
          "num_iterations" : 15148,
          "top_k" : 30, 
          "max_depth" : max_depth, 
          "num_leaves" : num_leaves, 
          "min_data_in_leaf" : min_data_in_leaf, 
          "learning_rate" : 0.02,
          "bagging_fraction" : 0.7, 
          "bagging_seed" : 3,
          "bagging_freq" : 5, 
          "feature_fraction" : 0.5, 
          "num_threads" : 4
         }
dataset_params = {"max_bin" : 200, 
                  "min_data_in_bin" : 3} 



model = LGBMRegressor(**params)
#model = LGBMRegressor(min_child_samples=min_child_samples, n_estimators=n_estimators, learning_rate=learning_rate ,max_depth = max_depth, num_leaves = num_leaves, min_data_in_leaf = min_data_in_leaf)

In [30]:
fit_params={"early_stopping_rounds":150, 
            "eval_metric" : 'RMSE', 
            "eval_set" : [(X_val, y_val.reshape(-1))],
            'eval_names': ['valid'],
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': cat_vars
           }

In [31]:
model.fit(X_train, y_train.reshape(-1), **fit_params)

C:\Users\pedro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] num_threads is set=4, n_jobs=-1 will be ignored. Current value: num_threads=4
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


C:\Users\pedro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\pedro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[100]	valid's rmse: 0.0218855
[200]	valid's rmse: 0.0172241
[300]	valid's rmse: 0.0152695
[400]	valid's rmse: 0.0141639
[500]	valid's rmse: 0.0134994
[600]	valid's rmse: 0.0130321
[700]	valid's rmse: 0.012729
[800]	valid's rmse: 0.0124639
[900]	valid's rmse: 0.0122713
[1000]	valid's rmse: 0.0120771
[1100]	valid's rmse: 0.0119376
[1200]	valid's rmse: 0.0117478
[1300]	valid's rmse: 0.0116084
[1400]	valid's rmse: 0.0114823
[1500]	valid's rmse: 0.0113914
[1600]	valid's rmse: 0.0112964
[1700]	valid's rmse: 0.011198
[1800]	valid's rmse: 0.011125
[1900]	valid's rmse: 0.0110718
[2000]	valid's rmse: 0.0110126
[2100]	valid's rmse: 0.0109624
[2200]	valid's rmse: 0.0109215
[2300]	valid's rmse: 0.0108909
[2400]	valid's rmse: 0.0108454
[2500]	valid's rmse: 0.0108162
[2600]	valid's rmse: 0.0107903
[2700]	valid's rmse: 0.0107653
[2800]	valid's rmse: 0.0107434
[2900]	valid's rmse: 0.0107226
[3000]	valid's rmse: 0.0107087
[3100]	valid's rmse: 0.0106903
[3200]	valid's rmse: 0.0106708
[3300]	valid's rmse:

LGBMRegressor(bagging_fraction=0.7, bagging_freq=5, bagging_seed=3,
              boosting='gbdt', feature_fraction=0.5, learning_rate=0.02,
              max_depth=8, metric='rmse', min_data_in_leaf=20,
              num_iterations=15148, num_leaves=800, num_threads=4,
              objective='rmse', top_k=30)

# Métrica

$$
\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

In [32]:
model.score(X_val, y_val)

0.9251516470349543

In [33]:
if log_output:
    y_pred_train = np.exp(model.predict(X_train, verbose=1)*max_log_y)
    y_pred = np.exp(model.predict(X_val, verbose=1)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test, verbose=1)*max_log_y)
else:
    y_pred_train = model.predict(X_train, verbose=1)*y_std + y_mean
    y_pred = model.predict(X_val, verbose=1)*y_std + y_mean
    y_pred_test = model.predict(X_test, verbose=1)*y_std + y_mean

In [34]:
# Train
np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train))

0.06892762689029042

In [35]:
#0.0787547817613104
#0.07360962647765361

In [36]:
# Validación
np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))

0.11376692686385394

In [37]:
#0.12355046837823216
#0.11745998404616953

# Baseline

In [38]:
import pandas as pd
sample_csv = pd.read_csv('rossmann/sample_submission.csv')

In [39]:
stores_mean = {}
for store, g_df in df.groupby('Store'):
    stores_mean[store] = g_df[g_df['Sales'] > 0]['Sales'].mean()

In [40]:
df_test['Sales'] = df_test['Store'].apply(stores_mean.get)
df_test.loc[df_test['Open'] == 0, 'Sales'] = 0

In [41]:
df_test[['Store', 'Sales']].head(10)

,Store,Sales
0,0,4759.096031
1,2,6942.568678
2,6,8817.050891
3,7,5539.358418
4,8,6562.337612
5,9,5568.420918
6,10,8030.977041
7,11,7589.598214
8,12,5034.747182
9,13,5508.567394


In [42]:
df_test[df_test['Open'] == 0][['Store', 'Sales']].head()

,Store,Sales
543,702,0.0
676,878,0.0
840,1096,0.0
1399,702,0.0
1532,878,0.0


In [43]:
sample_csv['Sales'] = df_test['Sales']

In [44]:
sample_csv.to_csv(f'submision_baseline.csv', index=False)

In [45]:
sample_csv.head()

,Id,Sales
0,1,4759.096031
1,2,6942.568678
2,3,8817.050891
3,4,5539.358418
4,5,6562.337612


# Sumbit a la competición

In [47]:
sample_csv = pd.read_csv('rossmann/sample_submission.csv')
sample_csv['Sales'] = y_pred_test
sample_csv.head()

sample_csv.to_csv(f'submision_LGBM_muchos_param_{log_output}-{min_child_samples}-{n_estimators}-{learning_rate}-{max_depth}-{num_leaves}-.csv', index=False)